In [ ]:
1+1

In [1]:
1-1

0

In [38]:
1

1

In [19]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

from sklearn.metrics import confusion_matrix
from sklearn import metrics

In [20]:
tf.__version__, keras.__version__

('2.6.0', '2.6.0')

In [21]:
df = pd.read_csv('all_yrs_processed.csv')
df.head()

,ein,unrelbusinccd,initiationfees,grsrcptspublicuse,grsincmembers,grsincother,totcntrbgfts,totprgmrevnue,invstmntinc,txexmptbndsproceeds,...,nonpfreayr-1_8,nonpfreayr-1_9,nonpfreayr-1_11,nonpfreayr-1_12,nonpfreayr-1_13,nonpfreayr-1_14,nonpfreayr-1_15,y_term,y_liq,y_TL
0,910454080,0,0,0,0,0,935508,120315,7456,19969,...,0,0,0,0,0,0,0,0,0,0
1,111966978,0,0,0,0,0,0,5050020,153784,0,...,0,0,0,0,0,0,0,0,0,0
2,420660491,0,0,0,0,0,2945,11440,489,0,...,0,0,0,0,0,0,0,0,0,0
3,530152390,1,220425,468101,0,0,411058,2668044,29316,0,...,0,0,0,0,0,0,0,0,0,0
4,561547207,0,0,0,0,0,75,735052,783,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
df.shape

(266387, 545)

In [23]:
df = df.dropna()
print(df.shape)

y_cols = ['y_term', 'y_liq', 'y_TL']
X = df.drop(columns = y_cols).drop(columns = 'ein')
y = df[y_cols]

y1 = y['y_TL'] # y1 - y full/partial 
yt = y['y_term'] # yt - y term (full termination)
yl = y['y_liq']  # yl - y liquid (partial liquidation)

X.shape, y.shape

(228181, 545)


((228181, 541), (228181, 3))

In [24]:
cat_cols = X.nunique()[X.nunique() <=2].index
num_cols = X.nunique()[X.nunique() > 2].index

len(cat_cols), len(num_cols)

(211, 330)

In [25]:
# transfer to log df
numeric_df = X.apply(lambda x: np.log10(x) if np.issubdtype(x.dtype, np.number) else x)
numeric_df = numeric_df.replace(0,1)
numeric_df[np.isfinite(numeric_df)==0]=0
numeric_df.head()

/opt/conda/lib/python3.7/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.7/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


,unrelbusinccd,initiationfees,grsrcptspublicuse,grsincmembers,grsincother,totcntrbgfts,totprgmrevnue,invstmntinc,txexmptbndsproceeds,royaltsinc,...,nonpfreayr-1_5,nonpfreayr-1_6,nonpfreayr-1_7,nonpfreayr-1_8,nonpfreayr-1_9,nonpfreayr-1_11,nonpfreayr-1_12,nonpfreayr-1_13,nonpfreayr-1_14,nonpfreayr-1_15
1,0.0,0.000000,0.000000,0.0,0.0,0.000000,6.703293,5.186911,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1.0,0.000000,0.000000,0.0,0.0,6.294340,6.774271,4.483887,0.0,4.683299,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.000000,0.000000,0.0,0.0,0.000000,5.529430,1.913814,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,1.0,6.051747,5.339859,0.0,0.0,0.000000,6.483899,4.277128,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,1.0,0.000000,0.000000,0.0,0.0,6.930669,6.842122,3.616476,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
# Train-test split - reg
X_train, X_test, y_train, y_test = train_test_split(X, y1, test_size=0.3) # 70% training and 30% test
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((159726, 541), (68455, 541), (159726,), (68455,))

In [27]:
# Train-test split - log df
X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(numeric_df, y1, test_size=0.3) # 70% training and 30% test
X_train_log.shape, X_test_log.shape, y_train_log.shape, y_test_log.shape

((159726, 541), (68455, 541), (159726,), (68455,))

In [28]:
# scale data

means = np.mean(np.array(X_train[num_cols]), axis=0, keepdims=True)
stds = np.std(np.array(X_train[num_cols]), axis=0, keepdims=True)

X_train.loc[:,num_cols] = (X_train[num_cols]-means)/stds
X_test.loc[:,num_cols] = (X_test[num_cols]-means)/stds

means_log = np.mean(np.array(X_train_log[num_cols]), axis=0, keepdims=True)
stds_log = np.std(np.array(X_train_log[num_cols]), axis=0, keepdims=True)

X_train_log.loc[:,num_cols] = (X_train_log[num_cols]-means_log)/stds_log
X_test_log.loc[:,num_cols] = (X_test_log[num_cols]-means_log)/stds_log

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1884: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

In [ ]:
# to csv:

# X_train.append(X_test).to_csv('all_yrs_processed_andscaled.csv', index = False)

In [29]:
class FullyConnected():
  """ Eventually implement class """

  def __init__(self, model, X, y, bs, e, X_test,y_test, verbose = 1 ):         
    self.model , self.X , self.y , self.bs , self.e , self.X_test, self.y_test, self.verbose =  model, X, y, bs, e,X_test, y_test,  verbose
    

  def train(self): 
    # train/validation split
    self.X_train, self.X_valid, self.y_train, self.y_valid = train_test_split(self.X, self.y, test_size = 0.2) 
    print(self.X_train.shape, self.y_train.shape, self.X_valid.shape, self.X_test.shape)

    # fit model
    self.history = self.model.fit(x=self.X_train,
                        y=self.y_train,
                        batch_size=self.bs,
                        epochs=self.e,
                        verbose = self.verbose,
                        validation_data=(self.X_valid, self.y_valid))
    
    # Training results
    self.results = pd.DataFrame(self.history.history)
    print(self.model.summary())

  def eval(self):
    # Plot training history 
    self.results.plot()
    plt.title("Errors over time")
    plt.xlabel('Epochs')
    plt.show()
    
    self.model.evaluate(self.X_test, self.y_test)

    # Out-of-sample prediction
    
#     return model
    
#     self.y_hat = self.model.predict(self.X_test)
    
#     print(self.y_hat.shape)
#     print(self.y_test.shape)
#     self.accuracy = np.sum(self.y_hat == self.y_test) / len(self.y_hat)

    
#     print(self.y_test[0])
#     print(self.y_hat[0])
    
#     self.cm = confusion_matrix(self.y_test, self.y_hat)
#     print(self.cm)
    

    # Errors
#     self.errs = (self.y_hat - self.y_test)
#     self.errs.plot.hist(bins = np.arange(-4,4,0.5) )
#     plt.title('Distribution of Absolute Errors')
#     plt.show()

#     print(f"Test Accuracy: { self.accuracy.values[0]}  \n Test F1: {self.f1}")
    
#     return self.results
#     return [self.results.iloc[-1]['root_mean_squared_error'], self.results.iloc[-1]['val_root_mean_squared_error'], self.rmse.values[0] ]

# Initialize Results Dictionary
# res_dict = {}

# fully connected nn



In [13]:
# fit model
learning_rate=0.0001
epochs=25
my_batch_size=300

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=10, activation="relu")),
model.add(tf.keras.layers.Dense(units=15, activation="relu")),
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss='binary_crossentropy',
            metrics=[tf.keras.metrics.AUC()])

2021-08-25 11:29:39.653844: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-25 11:29:39.653915: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-25 11:29:39.653949: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (python-20210724-160919): /proc/driver/nvidia/version does not exist
2021-08-25 11:29:39.654510: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [36]:
mo = FullyConnected(model,X_train, y_train, my_batch_size,epochs, X_test, y_test )

In [ ]:
mo.train()

(127780, 541) (127780,) (31946, 541) (68455, 541)
Epoch 1/30
426/426 [==============================] - 2s 5ms/step - loss: 0.0694 - auc: 0.6877 - val_loss: 0.0701 - val_auc: 0.6837
Epoch 2/30
399/426 [===========================>..] - ETA: 0s - loss: 0.0691 - auc: 0.6878

In [ ]:
mo.eval()

In [ ]:
# q = mo.eval()

In [26]:
q.predict(mo.X_test)

array([[0.00361684],
       [0.00779808],
       [0.00623566],
       ...,
       [0.01287872],
       [0.03143376],
       [0.01240116]], dtype=float32)

In [30]:
def build_model(learning_rate, sizes):
    model = tf.keras.models.Sequential()
    for s in sizes:
        model.add(tf.keras.layers.Dense(units=s, activation="relu"))
        
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                loss='binary_crossentropy',
                metrics=[tf.keras.metrics.AUC()])

    return model

def train_model(model, feature, label, val_feature, val_label, epochs, batch_size):
    history = model.fit(x=feature,
                      y=label,
                      validation_data=(val_feature, val_label),
                      batch_size=batch_size,
                      epochs=epochs)
    

    trained_weight = model.get_weights()[0]
    trained_bias = model.get_weights()[1]

    epochs = history.epoch
    hist = pd.DataFrame(history.history)


    return trained_weight, trained_bias, epochs, hist

In [ ]:
# fit model
learning_rate=0.0001
epochs=25
my_batch_size=300
hidden_sizes2 = [10,10, 2]
NN_2HL2 = build_model(learning_rate, hidden_sizes2)
trained_weight2, trained_bias2, epochs2, hist2 = train_model(NN_2HL2, X_input, y_input, X_val, y_val, epochs, my_batch_size)

In [ ]:
NN_2HL2.evaluate(X_test, y_test)

In [ ]:
hist2.plot()

In [31]:
# Train-valid split
X_input, X_val, y_input, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [32]:
def build_model(learning_rate, sizes):
    model = tf.keras.models.Sequential()
    for s in sizes:
        model.add(tf.keras.layers.Dense(units=s, activation="relu"))
        
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                loss='binary_crossentropy',
                metrics=[tf.keras.metrics.AUC()])

    return model

In [ ]:
# fit model
learning_rate=0.0001
epochs=30
my_batch_size=300
hidden_sizes3 = [20,15,10, 5]
NN_2HL3 = build_model(learning_rate, hidden_sizes3)
trained_weight3, trained_bias3, epochs3, hist3 = train_model(NN_2HL3, X_input, y_input, X_val, y_val, epochs, my_batch_size)

Epoch 1/30
426/426 [==============================] - 3s 4ms/step - loss: 0.3999 - auc_1: 0.5117 - val_loss: 0.1272 - val_auc_1: 0.5102
Epoch 2/30
426/426 [==============================] - 1s 3ms/step - loss: 0.0957 - auc_1: 0.5079 - val_loss: 0.0798 - val_auc_1: 0.5111
Epoch 3/30
426/426 [==============================] - 1s 3ms/step - loss: 0.0813 - auc_1: 0.5152 - val_loss: 0.0753 - val_auc_1: 0.5240
Epoch 4/30
426/426 [==============================] - 1s 3ms/step - loss: 0.0777 - auc_1: 0.5416 - val_loss: 0.0733 - val_auc_1: 0.5474
Epoch 5/30
426/426 [==============================] - 1s 3ms/step - loss: 0.0756 - auc_1: 0.5671 - val_loss: 0.0721 - val_auc_1: 0.5631
Epoch 6/30
222/426 [==============>...............] - ETA: 0s - loss: 0.0737 - auc_1: 0.5859

In [ ]:
NN_2HL3.evaluate(X_test, y_test)

In [ ]:
hist3.plot()

In [ ]:
# Hyperparameters:
lr = .01
epochs = 30
batch_size = 300

# Build Model:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=60, input_shape=(541,)))
model.add(tf.keras.layers.Dense(units=80))
model.add(tf.keras.layers.Dense(units=10))
model.add(tf.keras.layers.Dense(units=2, activation = 'sigmoid'))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), 
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.AUC()])

model.summary()



In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
y_test = np.asarray(y_test).astype('float32').reshape((-1,1))

In [ ]:
y_train.shape

In [ ]:

# fit model
history = model.fit(x=X_train,
                    y=y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose = True,
                    validation_split=0.2)